# 범주형 데이터 실수화 실습

## 1. 범주형 데이터 실수화

### 1) 필요 라이브러리 import

In [1]:
import pandas as pd   
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder 

### 2) 데이터 set 생성 및 확인

In [2]:
# 데이터 set 생성
x_train = pd.DataFrame(["남성", "여성", "남성", "여성", "남성", "여성"], columns = ["성별"])

# 데이터 확인
x_train.head(3)

,성별
0,남성
1,여성
2,남성


In [3]:
# 데이터 프레임 정보 확인
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   성별      6 non-null      object
dtypes: object(1)
memory usage: 176.0+ bytes


In [4]:
# 범주형 데이터 개수 확인
x_train['성별'].value_counts()

남성    3
여성    3
Name: 성별, dtype: int64

### 3) 레이블 인코딩
- 문자를 0부터 시작하는 정수형 숫자로 변환

In [5]:
# 레이블 인코더 생성
encoding = LabelEncoder()

# x_train 데이터를 이용하여 피팅하고 레이블 숫자로 변환
encoding.fit(x_train["성별"])
x_train["성별_인코딩"] = encoding.transform(x_train["성별"])

### 4) 인코딩 결과 확인

In [6]:
# 인코딩 결과 확인
x_train

,성별,성별_인코딩
0,남성,0
1,여성,1
2,남성,0
3,여성,1
4,남성,0
5,여성,1


In [7]:
# 클래스 확인
print(list(encoding.classes_))  

# 인코딩 값으로 문자값 확인
print(list(encoding.inverse_transform([1, 0])))  

['남성', '여성']
['여성', '남성']


## 2. One-Hot Encoding

### 1) 데이터 확인(실수화한 데이터 사용)

In [8]:
x_train.head(2)

,성별,성별_인코딩
0,남성,0
1,여성,1


### 2) One-Hot Encoding

In [9]:
# 원핫인코더 생성
one_encoding = OneHotEncoder(sparse=False)

# x_train 데이터를 이용하여 피팅
one_encoding.fit(x_train[["성별"]])

# 가변환값 변환
one_encoding.transform(x_train[["성별"]])

# one_encoding.fit_transform(x_train[["성별"]])

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [10]:
# 가변환된 피쳐 확인
print(one_encoding.get_feature_names())

['x0_남성' 'x0_여성']


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### 3) 인코딩 결과 확인

In [11]:
# 가변환된 값을 x_train_one 데이터프레임으로 저장
x_train_one = pd.DataFrame(one_encoding.transform(x_train[['성별']]), columns= ['성별0', '성별1'])

# x_train 데이터와 x_train_one의 가변환된 속성 합침
x_train = pd.concat([x_train, x_train_one], axis=1)

# 결과 확인
x_train.head(3)

,성별,성별_인코딩,성별0,성별1
0,남성,0,1.0,0.0
1,여성,1,0.0,1.0
2,남성,0,1.0,0.0


# 텍스트 데이터 실수화 실습

## 1. 단어의 빈도수를 카운트하여 단어를 수치화

### 1) 필요 라이브러리 import

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

### 2) 데이터 set 생성 및 확인

In [13]:
corpus = [
    '청년 인재 개발 양성 과정',
    '인공지능 청년 양성',
    '미래 인공지능 데이터 대한민국',
    '데이터 원유 기술사 청년 개발'
]

In [14]:
corpus

['청년 인재 개발 양성 과정', '인공지능 청년 양성', '미래 인공지능 데이터 대한민국', '데이터 원유 기술사 청년 개발']

In [15]:
type(corpus)

list

### 3) 텍스트 실수화: 단어의 빈도수를 카운트

In [16]:
# 카운트 벡터라이저 생성
count_vect = CountVectorizer()

# 단어 카운트
count_vect.fit(corpus)

# 단어 확인
count_vect.vocabulary_

{'청년': 10,
 '인재': 9,
 '개발': 0,
 '양성': 6,
 '과정': 1,
 '인공지능': 8,
 '미래': 5,
 '데이터': 4,
 '대한민국': 3,
 '원유': 7,
 '기술사': 2}

In [17]:
# 단어 벡터화한 값을 array로 변환하여 확인
count_vect.transform(corpus).toarray()

array([[1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1]], dtype=int64)

In [18]:
# 단어 벡터화
features = count_vect.transform(corpus)
features

<4x11 sparse matrix of type '<class 'numpy.int64'>'
	with 17 stored elements in Compressed Sparse Row format>

### 4) 문서단어행렬로 변환

In [19]:
# 속성 이름만 반환
vocab = count_vect.get_feature_names()
vocab

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['개발', '과정', '기술사', '대한민국', '데이터', '미래', '양성', '원유', '인공지능', '인재', '청년']

In [20]:
# 문서단어행렬을 데이터프레임으로 변환
DTM = pd.DataFrame(features.toarray(), columns = vocab).head()
DTM

,개발,과정,기술사,대한민국,데이터,미래,양성,원유,인공지능,인재,청년
0,1,1,0,0,0,0,1,0,0,1,1
1,0,0,0,0,0,0,1,0,1,0,1
2,0,0,0,1,1,1,0,0,1,0,0
3,1,0,1,0,1,0,0,1,0,0,1


In [21]:
# 문서단어행렬(DTM)에 일치하는 단어 확인
count_vect.transform(['기술사 대한민국 인재 만세']).toarray()

array([[0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0]], dtype=int64)

## 2. 단어의 빈도수를 단어들마다 중요한 정도를 가중치로 주어 수치화

### 1) 필요 라이브러리 import

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

### 2) 데이터 set 생성 및 확인

In [23]:
corpus

['청년 인재 개발 양성 과정', '인공지능 청년 양성', '미래 인공지능 데이터 대한민국', '데이터 원유 기술사 청년 개발']

### 3) 텍스트 실수화: TF-IDF 방식

In [24]:
# TF-IDF 벡터라이저 생성
tfid = TfidfVectorizer()

# 단어 카운트
tfid.fit(corpus)

# 단어사전 확인
tfid.vocabulary_

{'청년': 10,
 '인재': 9,
 '개발': 0,
 '양성': 6,
 '과정': 1,
 '인공지능': 8,
 '미래': 5,
 '데이터': 4,
 '대한민국': 3,
 '원유': 7,
 '기술사': 2}

In [25]:
# 단어 벡터화한 값을 array로 변환하여 확인
tfid.transform(corpus).toarray()

array([[0.41263976, 0.52338122, 0.        , 0.        , 0.        ,
        0.        , 0.41263976, 0.        , 0.        , 0.52338122,
        0.33406745],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.61366674, 0.        , 0.61366674, 0.        ,
        0.49681612],
       [0.        , 0.        , 0.        , 0.55528266, 0.43779123,
        0.55528266, 0.        , 0.        , 0.43779123, 0.        ,
        0.        ],
       [0.41263976, 0.        , 0.52338122, 0.        , 0.41263976,
        0.        , 0.        , 0.52338122, 0.        , 0.        ,
        0.33406745]])

In [26]:
# 단어 벡터화
features_idf = tfid.transform(corpus)
features_idf

<4x11 sparse matrix of type '<class 'numpy.float64'>'
	with 17 stored elements in Compressed Sparse Row format>

### 4) 문서단어행렬로 변환

In [27]:
# 속성 이름만 반환
vocab_idf = count_vect.get_feature_names()
vocab_idf

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['개발', '과정', '기술사', '대한민국', '데이터', '미래', '양성', '원유', '인공지능', '인재', '청년']

In [28]:
# 문서단어행렬을 데이터프레임으로 변환
DTM_idf = pd.DataFrame(features_idf.toarray(), columns = vocab_idf).head()
DTM_idf

,개발,과정,기술사,대한민국,데이터,미래,양성,원유,인공지능,인재,청년
0,0.41264,0.523381,0.000000,0.000000,0.000000,0.000000,0.412640,0.000000,0.000000,0.523381,0.334067
1,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.613667,0.000000,0.613667,0.000000,0.496816
2,0.00000,0.000000,0.000000,0.555283,0.437791,0.555283,0.000000,0.000000,0.437791,0.000000,0.000000
3,0.41264,0.000000,0.523381,0.000000,0.412640,0.000000,0.000000,0.523381,0.000000,0.000000,0.334067
